In [2]:
import numpy as np
import pandas as pd
import csv
import matplotlib.pyplot as plt
import os
from PIL import Image
%matplotlib inline

In [6]:
def trainPics(path):
    path = os.getcwd() + path 
    
    trainFile= open("trainData.csv","w+")
    trainFile.write("Red, Green, Blue, isSkin\n")
    for root, dirs, files in os.walk(path + 'Mask\\'):
        for filename in files:
            #print(filename)
            if filename.endswith(".bmp") : 
                im = Image.open(path + 'Mask\\'+ filename, 'r')
                filename, extension = filename.split('.')
                imOriginal = Image.open(path + filename+'.jpg', 'r') 
                pix_val_mask = list(im.getdata())
                pix_val = list(imOriginal.getdata())
                pixelCount = len(pix_val)
                
                for i in range(pixelCount) :
                    #print(pix_val[i])
                    trainFile.write(str(pix_val[i][0])+', '+str(pix_val[i][1])+', '+str(pix_val[i][2])+', '+ ('0' if (pix_val_mask[i][0]>=230 and pix_val_mask[i][1]>=230 and pix_val_mask[i][2]>=230) else '1') + '\n')
                   
        else:
            continue
            
    trainFile.close()
        

path = '\ibtd\\'
#trainPics(path)

In [12]:
def calculateProbabilities(path):
    
    data = pd.read_csv(path)
    #data.info()
    pixelCount = np.zeros((256, 256, 256))
    skinCount = np.zeros((256, 256, 256))
    nonSkinCount = np.zeros((256, 256, 256))
    
    
    for index, item in data.iterrows():
        r = item['Red']
        g = item['Green']
        b = item['Blue']
        isSkin = item['isSkin']
        #print(r,g,b,isSkin)
        
        
        pixelCount[r][g][b] = pixelCount[r][g][b] + 1 
        if isSkin:
            skinCount[r][g][b] = skinCount[r][g][b] + 1
        else:
            nonSkinCount[r][g][b] = nonSkinCount[r][g][b] + 1
            
            
        
    probFile = open("rgbProb.csv","w+")
    probFile.write("Red,Green,Blue,skinProb,nonSkinProb\n")
    
    for i in range(256):
        for j in range(256):
            for k in range(256):
                if pixelCount[i][j][k] == 0 :
                    probFile.write(str(i) +','+ str(j) +','+ str(k) +','+ '0' +','+ '1' + '\n')
                else:
                    skinProb = skinCount[i][j][k]/pixelCount[i][j][k]
                    nonSkinProb = nonSkinCount[i][j][k]/pixelCount[i][j][k]
                    probFile.write(str(i) +','+ str(j) +','+ str(k) +','+ str(skinProb) +','+ str(nonSkinProb) + '\n')
    
    probFile.close()
    
#calculateProbabilities('trainData.csv')